In [48]:
import pandas as pd

# Ezzel ne foglalkozz, adat betoltes
# X O
def load_and_process_measurements2(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)

    df['label'] = -1
    current_label = -1
    in_measurement = False

    for idx, row in df.iterrows():
        if (row == 'X').any():
            current_label += 1
            in_measurement = True
            df.at[idx, 'label'] = current_label
        elif (row == 'O').any():
            df.at[idx, 'label'] = current_label
            in_measurement = False
        elif in_measurement:
            df.at[idx, 'label'] = current_label
        else:
            df.at[idx, 'label'] = -1

    df = df[~df.apply(
        lambda row: row.astype(str).eq('X').any() or row.astype(str).eq('O').any(), axis=1)]

    df = df[df['label'] != -1]
    df.reset_index(inplace=True, drop=True)

    df = df.astype(float)

    df['time'] = df['Timestamp'] - df['Timestamp'].iloc[0]

    df['relative_time'] = df.groupby('label')['Timestamp'].transform(lambda x: x - x.iloc[0])

    df = df.drop(columns=['Timestamp'])

    return df


In [49]:
import pandas as pd

# Csak O
def load_and_process_measurements(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)

    df['label'] = -1
    current_label = -1
    in_measurement = False

    for idx, row in df.iterrows():
        if (row == 'O').any():
            current_label += 1
            in_measurement = True
        if in_measurement:
            df.at[idx, 'label'] = current_label
        else:
            df.at[idx, 'label'] = -1

    df = df[~df.apply(
        lambda row: row.astype(str).eq('X').any() or row.astype(str).eq('O').any(), axis=1)]

    df = df[df['label'] != -1]
    df.reset_index(inplace=True, drop=True)

    df = df.astype(float)

    df['time'] = df['Timestamp'] - df['Timestamp'].iloc[0]

    df['relative_time'] = df.groupby('label')['Timestamp'].transform(lambda x: x - x.iloc[0])

    df = df.drop(columns=['Timestamp'])

    return df


In [50]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Ha ezen valtoztatsz az egesz el fog baszodni(hacsak mindenhol nem igazitod hozza) es a modellt is ujra kell trainelni.
# De am csak a datat dolgozza fel
def prepare_sequence_data(df: pd.DataFrame, max_len: int = None):
    feature_cols = df.columns.difference(['label', 'time'])
    sequences = []
    labels = []
    
    grouped = df.groupby('label')

    for _, group in grouped:
        seq = group[feature_cols].values
        sequences.append(seq)

    if max_len is None:
        max_len = max(len(seq) for seq in sequences)

    sequences_padded = pad_sequences(sequences, maxlen=max_len, padding='post', dtype='float32')
    labels = np.array(labels)

    return sequences_padded

In [51]:
# Itt írd át a pathet arra a .csv fajlra ahol vannak a measurementjeid. Nagyon fontos hogy a .csv fajl ugyanugy van generalva mint amit adtal,
# es egyik mérés se lehet tobb soros mint a correct.csv leghosszabb merese(65 sor).
path = r'message.csv'
# Na ez a measurement number azt tudja hogy ha beadod neki a csvt amiben benne vannak a measurementek(minimum 1 de lehet tobb is) akk ugye 
# a preprocesselos resz beszamozza a measurementeket 0 tol kezdve(tehat az elso Xes sortol az elso Oos sorig van a 0. measurement) es ez a szam hatarozza meg
# hogy epp hanyadik measurementrol akarod eldonteni hogy kor vagy nem. A measurement number nem lehet nagyobb
# mint ahany meresed van a betoltott csvben.(sztem logikus)

# Valszeg jo de írd át ha szar/masik mappaba akarod tenni
model_path = r'best_model.h5'
# Ha átírod új model kell
max_rows = 150

new_data = load_and_process_measurements(path)
print('Correct circles length:', new_data.shape[0], 'Number of measurements:', new_data['label'].nunique(), 'Avg measurement length:', new_data.shape[0] / new_data['label'].nunique())
print('Longest measuremet:', new_data['label'].value_counts().max(), 'Shortest:', new_data['label'].value_counts().min())


measurement_numbers = [i for i in range(new_data['label'].nunique())] # Összes measurement
print('measurement_number in range of data length:', max(measurement_numbers) < new_data['label'].nunique())

if max(measurement_numbers) >= new_data['label'].nunique(): 
    raise ValueError("Figyeljel ocskos")
new_data.head()

Correct circles length: 449 Number of measurements: 3 Avg measurement length: 149.66666666666666
Longest measuremet: 217 Shortest: 44
measurement_number in range of data length: True


,linear_accelerationX,linear_accelerationY,linear_accelerationZ,magX,magY,magZ,gyroX,gyroY,gyroZ,accX,accY,accZ,label,time,relative_time
0,0.125090,0.292730,-0.279212,19.250488,12.744141,-21.752930,-10.925293,68.786621,20.935059,0.067261,-0.217773,-1.002197,0.0,0.000000,0.000000
1,0.067798,0.255958,-0.295246,18.603516,9.606934,-21.545410,59.020996,5.126953,32.165527,0.011108,-0.250244,-1.023193,0.0,0.089924,0.089924
2,0.146873,0.482348,-0.308014,17.724609,7.299805,-20.922852,58.410645,16.967773,57.861328,0.092651,-0.015747,-1.041138,0.0,0.135143,0.135143
3,0.062209,0.507510,-0.348663,17.419434,4.992676,-21.276855,35.827637,3.295898,-6.774902,0.009033,0.017944,-1.087646,0.0,0.179995,0.179995
4,0.091434,0.407161,-0.332815,17.346191,3.991699,-21.960449,14.465332,18.676758,-11.230469,0.039795,-0.075562,-1.077393,0.0,0.225114,0.225114


In [52]:
prepared_data = prepare_sequence_data(new_data, max_len=max_rows)

In [53]:
# Ehhez mar lusta voltam szval szar chatgpt kod de mukodnie kene
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model(model_path)

# Assume you have your processed data loaded:
# X_correct, y_correct = prepare_sequence_data(correct, label_value=1)

preds = []
for i in range(len(prepared_data)):
    if i in measurement_numbers:
        prediction = model.predict(np.expand_dims(prepared_data[i], axis=0))[0][0]
        print(f"{i}th prediction:", "Correct" if prediction >= 0.5 else "Incorrect")
        print("Confidence:", prediction)
        preds.append(prediction)


1/1 [==============================] - 0s 89ms/step
0th prediction: Incorrect
Confidence: 0.053705774
1/1 [==============================] - 0s 32ms/step
1th prediction: Incorrect
Confidence: 0.48465082
1/1 [==============================] - 0s 27ms/step
2th prediction: Correct
Confidence: 0.9999539


In [54]:
prepared_data.shape

(3, 150, 13)